In [31]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
from annotation_utils import get_synset_first, tag2pos
import annotation_utils as anno_utils
from collections import defaultdict

In [25]:
dataset = "flickr30k"

In [36]:
regiondf = pd.read_json("../data/flickr30k_regions_distractors.json.gz", compression='gzip', orient='split')

In [37]:
regiondf['ndistractors'] = regiondf.distractor_cats.apply(lambda x: len(x))

In [38]:
regiondf.iloc[100:200]

,distractor_cats,distractor_ids,distractor_names,image_id,names,region_cat,region_id,ndistractors
100,"[person, all, person, all, all, person, person...","[30604, 30605, 30607, 30608, 30610, 30611]","[toddler.n.01, younger.s.01, toddler.n.01, you...",2136080450,[shirt.n.01],[artifact],30609,18
101,"[person, all, person, all, all, person, person...","[30604, 30605, 30607, 30608, 30609, 30611]","[toddler.n.01, younger.s.01, toddler.n.01, you...",2136080450,[image.n.01],[cognition],30610,18
102,"[person, all, person, all, all, person, person...","[30604, 30605, 30607, 30608, 30609, 30610]","[toddler.n.01, younger.s.01, toddler.n.01, you...",2136080450,[camera.n.01],[artifact],30611,18
103,"[group, group, group, artifact, person, person...","[113781, 113764, 113767, 113781, 113764, 11377...","[work_force.n.01, work_force.n.01, people.n.01...",3372351555,"[man.n.01, man.n.01, man.n.01]","[person, person, person]",113763,27
104,"[person, person, person, group, group, artifac...","[113763, 113764, 113767, 113781, 113764, 11377...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,[work_force.n.01],[group],113781,29
105,"[person, person, person, group, artifact, pers...","[113763, 113781, 113767, 113781, 113764, 11377...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,"[work_force.n.01, people.n.01]","[group, group]",113764,28
106,"[person, person, person, group, group, group, ...","[113763, 113781, 113764, 113781, 113764, 11377...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,"[load.n.01, woman.n.01, woman.n.01, person.n.01]","[artifact, person, person, Tops]",113767,26
107,"[person, person, person, group, group, group, ...","[113763, 113781, 113764, 113767, 113781, 11376...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,[one.n.01],[quantity],113777,29
108,"[person, person, person, group, group, group, ...","[113763, 113781, 113764, 113767, 113781, 11376...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,"[bag.n.01, bag.n.01]","[artifact, artifact]",113765,28
109,"[person, person, person, group, group, group, ...","[113763, 113781, 113764, 113767, 113781, 11376...","[man.n.01, man.n.01, man.n.01, work_force.n.01...",3372351555,"[shirt.n.01, shirt.n.01]","[artifact, artifact]",113766,28


In [39]:
regiondf.iloc[200:500][["names","distractor_names"]]

,names,distractor_names
200,[shirt.n.01],"[woman.n.01, lady.n.01, woman.n.01, woman.n.01..."
201,"[crowd.n.01, people.n.01, number.n.01, people....","[two.n.01, populace.n.01, girl.n.01, two.n.01,..."
202,[two.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
203,[populace.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
204,[girl.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
205,[balloon.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
206,[tent.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
207,[summer.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
208,[baseball.n.01],"[crowd.n.01, people.n.01, number.n.01, people...."
209,"[clown.n.01, clown.n.01, clown.n.01, clown.n.0...","[balloon.n.01, balloon.n.01, balloon.n.01, bal..."


In [40]:
regiondf.iloc[499]

distractor_cats     [all, person, person, person, person, artifact...
distractor_ids      [153501, 153502, 153503, 153505, 153507, 15350...
distractor_names    [young.a.01, male_child.n.01, male_child.n.01,...
image_id                                                    405405965
names                            [girl.n.01, toddler.n.01, girl.n.01]
region_cat                                   [person, person, person]
region_id                                                      153504
ndistractors                                                       18
Name: 499, dtype: object